In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# for better plots
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')


from sklearn.cross_validation import KFold
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBM

In [2]:
secom = pd.read_table('secom.data', header=None, delim_whitespace=True)
Y = pd.read_table('secom_labels.data', header=None, usecols=[0], squeeze=True, delim_whitespace=True)

print('Temos {} linhas/exemplos e {} colunas/variáveis.'\
      .format(secom.shape[0], secom.shape[1]))
print('A classe majoritária possui {} exemplos e a minoritária {}.'\
      .format(Y[Y == -1].size, Y[Y == 1].size))

Temos 1567 linhas/exemplos e 590 colunas/variáveis.
A classe majoritária possui 1463 exemplos e a minoritária 104.


# Preprocessamento

###  Remova colunas que tem mais que 30% de dados faltantes. 

In [3]:
# vamos deixar 1096 valores por coluna, isto é, 70% de 1567, 
secom_30 = secom.dropna(thresh=1096, axis=1)
print('Restaram %d colunas.' % secom_30.shape[1])

Restaram 558 colunas.


### Remova as linhas que ainda tem dados faltantes. 

In [4]:
# transformar os dados do secom_30 em numpy-array
X = secom_30.as_matrix(columns=None)

# reshape nos labels para concatenar com X
Y = Y.reshape(1567,1)

# verificando as dimensões
print('Secom formato (com linhas com dados faltantes): %r.\nFormato dos respectivos Labels: %r.' % (X.shape, Y.shape))

# concatenar os labels Y e X. Assim, quando excluirmos as linhas com dados faltantes, 
# o conjunto de labels será mantido. 
XY = np.concatenate((X,Y), axis = 1)

# verificando a dimensão da concatenação, + 1 coluna (conjunto de labels)
print('Dados concatenados: (Secom + Labels): {}.'.format(XY.shape))

# excluindo as linhas que possuem algum dado faltante
df_non_null = XY[~np.isnan(XY).any(axis=1)]
print('Linhas com dados faltantes removidas. Novo número de linhas: %d.'\
      % df_non_null.shape[0])

# separando o data-frame das Labels
secom  = df_non_null[:, :558]
print('Novo formato do Secom (desconcatenado das Labels): {}.'.format(secom.shape))

labels = df_non_null[:, 558:559].reshape(978,)
# labels = df_non_null[:, 558:559]
print('Novo formato das Labels: {}.'.format(labels.shape))


Secom formato (com linhas com dados faltantes): (1567, 558).
Formato dos respectivos Labels: (1567, 1).
Dados concatenados: (Secom + Labels): (1567, 559).
Linhas com dados faltantes removidas. Novo número de linhas: 978.
Novo formato do Secom (desconcatenado das Labels): (978, 558).
Novo formato das Labels: (978,).


### Finalmente padronize as colunas para media 0 e desvio padrao 1.

In [5]:
from sklearn.preprocessing import scale

# scale() fornece média zero das colunas e desvio padrão = 1
std_secom = scale(secom)

# PCA

### Vamos encontrar 80% da variância e transformar os Dados. 


In [6]:
# função PCA com sklearn
def doPCA(x): # x é o argumento do número de componentes do PCA
    from sklearn.decomposition import PCA
    pca = PCA(n_components=x)
    pca.fit(std_secom)
    return pca



In [7]:
# executando o PCA com 558 componentes.
pca = doPCA(std_secom.shape[1]) 

# os elementos em _var contém a total variância.
_var = pca.explained_variance_ratio_ 

In [8]:
# armazena a variância.
requested_var = 0

# 80%  é a variância requisitada.
lim = 0.80

# iterações para encontrar a quantidade de dimensões que mantém
# 80% da variância
for i in range (len(_var)):
    requested_var += _var[i]
    if requested_var >= lim:
        element = i
        requested_var -= _var[i] # para ajuste do indice
        break

print('Com %d elementos temos a variância requisitada.' % (element))

# repetindo o PCA com 80% da variância
pca = doPCA(element) # element == 84 componentes

# prints
print('The %d principal components:' % (element))
np.set_printoptions(precision=3)


Com 84 elementos temos a variância requisitada.
The 84 principal components:


### Para 80% da variância é necessário manter 84 elementos. 

#### Vamos transformar os dados para fazer o KNN.

In [9]:
secom_PCA_80 = pca.transform(std_secom)

# formato dos dados com 80% da variância
print('Dados PCA com 80%, formato: {}.'\
      .format(secom_PCA_80.shape))

Dados PCA com 80%, formato: (978, 84).


# KNN com 5-fold externo e 3-fold interno

In [24]:
# acurácia inicial
acc = 0

# separa os dados transformados  secom_PCA_80 em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_k, best_acc = 0,0
        
    for k in [1, 5, 11, 15, 21, 25]:
        new_acc = 0
        
        for train_hp, test_hp in kf_3:
            # define o knn            
            knn = KNN(n_neighbors=k)
            # treina secom_PCA_80
            knn.fit(secom_PCA_80[train][train_hp], labels[train][train_hp]) 
            # avalia a acurária 
            new_acc += knn.score(secom_PCA_80[train][test_hp], labels[train][test_hp])
        
        # acurácia média do hiperparâmetro k
        new_acc = new_acc/3
        
        # verifica se acurácia do hiperparametro k atual é a melhor
        if (new_acc > best_acc):
            best_acc = new_acc
            best_k = k
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #{}: o melhor k é: {}'.format(count, best_k))
    
    # ao fim do loop, calcula a acurácia para o melhor k
    knn_out = KNN(n_neighbors=best_k)
    
    # treina com os valores do 5-fold externo
    knn_out.fit(secom_PCA_80[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*knn_out.score(secom_PCA_80[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc += knn_out.score(secom_PCA_80[test], labels[test])

acc /= 5    
print('\nAcurácia do Método: {}%'.format(acc*100))


Em fold #1: o melhor k é: 5
Acurácia do Fold = 84.6938775510204%
Em fold #2: o melhor k é: 5
Acurácia do Fold = 95.40816326530613%
Em fold #3: o melhor k é: 11
Acurácia do Fold = 97.44897959183673%
Em fold #4: o melhor k é: 5
Acurácia do Fold = 95.38461538461539%
Em fold #5: o melhor k é: 5
Acurácia do Fold = 92.3076923076923%

Acurácia do Método: 93.0486656200942%


# SVM
### Para o SVM RBF teste para C=2**(-5), 2**(0), 2**(5), 2**(10) e gamma= 2**(-15) 2**(-10) 2**(-5) 2**(0) 2**(5).

In [11]:
# acurácia inicial
acc = 0

# separa os dados transformados  std_secom em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_c, best_gamma, best_acc = 0,0,0
        
    for c in [2**-5, 2**0, 2**5, 2**10]:
        for gamma in [2**-15, 2**-10, 2**-5, 2**0, 2**5]:
            new_acc = 0
            for train_hp, test_hp in kf_3:
                # define o SVM classificador svc
                svc = SVC(C=c, kernel='rbf', gamma=gamma)
                # treina std_secom
                svc.fit(std_secom[train][train_hp], labels[train][train_hp]) 
                # avalia a acurária 
                new_acc += svc.score(std_secom[train][test_hp], \
                                     labels[train][test_hp])
        
            # acurácia média do hiperparâmetro k
            new_acc = new_acc/3
        
            # verifica se acurácia do hiperparametro k atual é a melhor
            if (new_acc > best_acc):
                best_acc = new_acc
                best_c = c
                best_gamma = gamma
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #%d: o melhor c é: %.6f e gamma: %.6f'
           % (count, best_c, best_gamma))
    
    # ao fim do loop, calcula a acurácia para o melhor c e gamma
    svc_out = SVC(C=best_c, kernel='rbf', gamma=best_gamma)
    
    # treina com os valores do 5-fold externo
    svc_out.fit(std_secom[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*svc_out.score(std_secom[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc += svc_out.score(std_secom[test], labels[test])

acc /= 5    
print('\nAcurácia do Método: {}%'.format(acc*100))


Em fold #1: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 84.6938775510204%
Em fold #2: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 95.91836734693877%
Em fold #3: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 97.44897959183673%
Em fold #4: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 95.38461538461539%
Em fold #5: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 92.3076923076923%

Acurácia do Método: 93.15070643642073%


# Rede Neural
###  Para a rede neural, teste com 10, 20, 30 e 40 neuronios na camada escondida.

In [25]:
# acurácia inicial
acc = 0

# separa os dados transformados  std_secom em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_neur, best_acc = 0,0
        
    for neur in [10, 20, 30, 40]:
        new_acc = 0
        
        for train_hp, test_hp in kf_3:
            # define o knn            
            nn = MLP(solver='lbfgs', alpha=1, hidden_layer_sizes=neur, random_state=1)
            # treina std_secom
            nn.fit(std_secom[train][train_hp], labels[train][train_hp]) 
            # avalia a acurária 
            new_acc += nn.score(std_secom[train][test_hp], labels[train][test_hp])
        
        # acurácia média do hiperparâmetro k
        new_acc = new_acc/3
        
        # verifica se acurácia do hiperparametro k atual é a melhor
        if (new_acc > best_acc):
            best_acc = new_acc
            best_neur = neur
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #{}: o melhor número de neurônios na camada escondida é: {}'.format(count, best_neur))
    
    # ao fim do loop, calcula a acurácia para o melhor k
    nn_out = MLP(solver='lbfgs', alpha=1, hidden_layer_sizes=best_neur, random_state=1)
    
    # treina com os valores do 5-fold externo
    nn_out.fit(std_secom[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*nn_out.score(std_secom[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc += nn_out.score(std_secom[test], labels[test])

acc /= 5    
print('\nAcurácia do Método: {}%'.format(acc*100))


Em fold #1: o melhor número de neurônios na camada escondida é: 40
Acurácia do Fold = 83.16326530612244%
Em fold #2: o melhor número de neurônios na camada escondida é: 30
Acurácia do Fold = 91.3265306122449%
Em fold #3: o melhor número de neurônios na camada escondida é: 20
Acurácia do Fold = 95.91836734693877%
Em fold #4: o melhor número de neurônios na camada escondida é: 30
Acurácia do Fold = 93.33333333333333%
Em fold #5: o melhor número de neurônios na camada escondida é: 30
Acurácia do Fold = 90.25641025641026%

Acurácia do Método: 90.79958137100995%


# Random Forest 

### Para o RF, teste com n_featrues = 10, 15, 20, 25 e ntrees = 100, 200, 300 e 400

In [26]:
# acurácia inicial
acc = 0

# separa os dados transformados  std_secom em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_n_features, best_n_features, best_acc = 0,0,0
        
    for n_features in [10, 15, 20, 25]:
        for n_trees in [100, 200, 300, 400]:
            new_acc = 0
            for train_hp, test_hp in kf_3:
                # define o SVM classificador svc
                rf = RF(max_features=n_features, max_depth=n_trees)
                # treina std_secom
                rf.fit(std_secom[train][train_hp], labels[train][train_hp]) 
                # avalia a acurária 
                new_acc += rf.score(std_secom[train][test_hp], \
                                     labels[train][test_hp])
        
            # acurácia média do hiperparâmetro k
            new_acc = new_acc/3
        
            # verifica se acurácia do hiperparametro k atual é a melhor
            if (new_acc > best_acc):
                best_acc        = new_acc
                best_n_features = n_features
                best_n_trees    = n_trees
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #%d: o melhor n_feature é: %.6f e n_tree: %.6f'
           % (count, best_n_features, best_n_trees))
    
    # ao fim do loop, calcula a acurácia para o melhor c e gamma
    rf_out = RF(max_features=best_n_features, max_depth=best_n_trees)
    
    # treina com os valores do 5-fold externo
    rf_out.fit(std_secom[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*rf_out.score(std_secom[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc += rf_out.score(std_secom[test], labels[test])

acc /= 5    
print('\nAcurácia do Método: {}%'.format(acc*100))


Em fold #1: o melhor n_feature é: 10.000000 e n_tree: 200.000000
Acurácia do Fold = 84.6938775510204%
Em fold #2: o melhor n_feature é: 10.000000 e n_tree: 100.000000
Acurácia do Fold = 95.91836734693877%
Em fold #3: o melhor n_feature é: 10.000000 e n_tree: 300.000000
Acurácia do Fold = 97.44897959183673%
Em fold #4: o melhor n_feature é: 10.000000 e n_tree: 300.000000
Acurácia do Fold = 94.87179487179486%
Em fold #5: o melhor n_feature é: 10.000000 e n_tree: 100.000000
Acurácia do Fold = 92.3076923076923%

Acurácia do Método: 93.04814233385663%


# Gradient Boosting Machine
### Para o GBM (ou XGB) teste para numero de arvores = 30, 70, e 100, com learning rate de 0.1 e 0.05, e profundidade da arvore=5.

In [32]:
# acurácia inicial
acc = 0

# separa os dados transformados  std_secom em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_learning_rate, best_n_tree, best_acc = 0.1,0,0
        
    for ln in [0.1, 0.05]:
        for n_trees in [30, 70, 100]:
            new_acc = 0
            for train_hp, test_hp in kf_3:
                # define o GBM classificador 
                gbm = GBM(n_estimators=n_trees, learning_rate=ln, max_depth=5)
                # treina std_secom
                rf.fit(std_secom[train][train_hp], labels[train][train_hp]) 
                # avalia a acurária 
                new_acc += rf.score(std_secom[train][test_hp], \
                                     labels[train][test_hp])
        
            # acurácia média do hiperparâmetro k
            new_acc = new_acc/3
        
            # verifica se acurácia do hiperparametro k atual é a melhor
            if (new_acc > best_acc):
                best_acc        = new_acc
                best_learning_rate = ln
                best_n_tree    = n_trees
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #%d: o melhor learning rate é: %.6f e n_tree: %.6f'
           % (count, best_n_tree, best_learning_rate))
    
    # ao fim do loop, calcula a acurácia para o melhor c e gamma
    gbm_out = GBM(n_estimators=best_n_tree, learning_rate=best_learning_rate)
    
    # treina com os valores do 5-fold externo
    gbm_out.fit(std_secom[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*gbm_out.score(std_secom[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc += gbm_out.score(std_secom[test], labels[test])

acc /= 5    
print('\nAcurácia do Método: {}%'.format(acc*100))


Em fold #1: o melhor learning rate é: 30.000000 e n_tree: 0.100000
Acurácia do Fold = 84.18367346938776%
Em fold #2: o melhor learning rate é: 100.000000 e n_tree: 0.100000
Acurácia do Fold = 93.36734693877551%
Em fold #3: o melhor learning rate é: 70.000000 e n_tree: 0.050000
Acurácia do Fold = 97.44897959183673%
Em fold #4: o melhor learning rate é: 30.000000 e n_tree: 0.050000
Acurácia do Fold = 94.35897435897435%
Em fold #5: o melhor learning rate é: 30.000000 e n_tree: 0.050000
Acurácia do Fold = 92.3076923076923%

Acurácia do Método: 92.33333333333334%
